# FUNZIONI + LIBRERIE

In [ ]:
# LIBRERIE
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import CategoricalDtype


In [ ]:
# CARICAMENTO DATI
def loadDataset(year):
    dataset_path = "../datasets/tennis/"+year+".xlsx"
    return pd.read_excel(dataset_path)



In [ ]:
# Mi serve per sapere l'encoding dei giocatori 
def playEncode(df):
    playerList=set(df["Winner"]) | set(df["Loser"]) # Trovo tutti i giocatori singoli
    
    # Formattazione dei Giocatori
    playerEncode = {}
    for index, player in enumerate(playerList):
        playerEncode[player] = index
        
    return playerEncode

In [ ]:
# Formattazione generale del dataset
def generalFormatting(dfCopy, playerEncode):
    
    df = dfCopy.copy()
    
    # Cambio nome delle colonne Winner/Loser
    dictCol = {'Winner' :"Player1", 'Loser':'Player2', 'WRank':'P1_Rank', 'LRank':'P2_Rank', 'WPts':'P1_Pts', 'LPts':'P2_Pts',
       'W1':'P1_S1', 'L1':'P2_S1', 'W2':'P1_S2', "L2":'P2_S2', 'W3':'P1_S3', 'L3':'P2_S3', 'W4':'P1_S4', 'L4':'P2_S4', 'W5':'P1_S5', "L5":'P2_S5', 'Wsets':'P1_sets',
       'Lsets':'P2_sets', 'B365W':'P1_B365', 'B365L':'P2_B365', 'EXW':'P1_EX', 'EXL':'P2_EX', 'LBW':'P1_LB', 'LBL':'P2_LB', 'PSW':'P1_PS',
       'PSL':'P2_PS', 'MaxW':'P1_Max', 'MaxL':'P2_Max', 'AvgW':'P1_Avg', 'AvgL':'P2_Avg'}

    df.rename(columns=dictCol, inplace=True)
    
    df.replace({"Player1":playerEncode, "Player2":playerEncode}, inplace = True)
    
    # Fill NA
    df.fillna(0, inplace = True)
    
    # One Hot Encoding
    df = pd.get_dummies(df, columns = ["Court", "Surface","Comment", "Best of"])
    
    # Encoding Tipo torneo con peso
    df["Series"]=df["Series"].astype('category')
    cats_to_order = ["ATP250", "ATP500", "Masters 1000", "Masters Cup", "Grand Slam"]
    covered_type = CategoricalDtype(categories=cats_to_order, ordered=True)
    df['Series'] = df['Series'].cat.reorder_categories(cats_to_order, ordered=True)
    df["Series"] = df["Series"].cat.codes
    
    # Encoding Round Torneo con peso
    df["Round"]=df["Round"].astype('category')
    cats_to_order = ["Round Robin", "1st Round",'2nd Round','3rd Round','4th Round','Quarterfinals', 'Semifinals', 'The Final']
    covered_type = CategoricalDtype(categories=cats_to_order, ordered=True)
    df['Round'] = df['Round'].cat.reorder_categories(cats_to_order, ordered=True)
    df["Round"] = df["Round"].cat.codes
    
    # Rimozione colonne influenti
    df.drop(["Location", "ATP"], axis =1, inplace=True)
    
    # Encoding Nome tornei
    lb_make = LabelEncoder()
    df["Tournament"] = lb_make.fit_transform(df["Tournament"])
    df["target"] = 0
    
    return df
    

In [ ]:
# Swap tra due valori
def swap (a,b):
    c=a
    a=b
    b=c
    return a,b

# Effetua lo swap dei valori di una riga
def swapP1P2 (row):
    copy = row.copy()
    swapCol=['Player1', 'Player2', 'P1_Rank', 'P2_Rank', 'P1_Pts', 'P2_Pts', 'P1_S1', 'P2_S1', 'P1_S2', 'P2_S2', 'P1_S3', 'P2_S3', 'P1_S4', 'P2_S4', 'P1_S5', 'P2_S5',
       'P1_sets', 'P2_sets', 'P1_B365', 'P2_B365', 'P1_EX', 'P2_EX', 'P1_LB', 'P2_LB', 'P1_PS', 'P2_PS', 'P1_Max', 'P2_Max', 'P1_Avg', 'P2_Avg']
    
    for i in range(0, int(len(swapCol)/2)):
        index=i*2
        aa = swapCol[index]
        bb = swapCol[index+1]
        copy[aa],copy[bb] = swap(copy[aa],copy[bb])
        copy["target"] = np.where (copy["target"] == 0, 1, 0)
        
    return copy
        
# Data un dataframe fa swap di colonne di righe random
def randomSwap(df):
    maxRow = df.shape[0]
    for i in range(0,1000):
        pos = np.random.randint(0,maxRow)
        df.loc[pos] = swapP1P2(df.loc[pos])

In [ ]:
# Crea una matrice di Giocatore X Giocatore segnando quante volte uno a vinto contro un altro
def playerDf(df_old, df_new):
    playerList=set(df_old["Winner"]) | set(df_old["Loser"]) | set(df_new["Winner"]) | set(df_new["Loser"]) # Trovo tutti i giocatori singoli
    
    newDf= pd.DataFrame(data = 0, index=playerList, columns=playerList) # Creo un dataframe per vedere chi vince con chi
    
    winLoseDf = df_old[["Winner","Loser"]] # Per popolarlo prendo chi vince e chi perde
    
    for index, row in winLoseDf.iterrows():       
        newDf.loc[row["Winner"],row["Loser"]]+=1
        
        
    return newDf

In [ ]:
# Mi serve per calcore il rateo di vittoria di un giocatore, se i giocatori non hanno giocato è 0.5
def rateo(p1, p2, wl):
    if wl.loc[p1, p2] == 0 and wl.loc[p2, p1] == 0:
        return 0.5
    else:
        return wl.loc[p1, p2] / (wl.loc[p1, p2]+wl.loc[p2,p1]) 

In [2]:
%%sh
pwd


/home/scodella/Scrivania/Stallio_and_Ollio/Web_intelligence/notebook/progetto


In [3]:
# Easy 
from function import helloWorld

helloWorld()

helloWorld
